In [5]:
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch
import time

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all" # all output p

/Users/abhinav/miniconda3/envs/trainllm/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
None of PyTorch, TensorFlow >= 2.0, or Flax have been found. Models won't be available and only tokenizers, configuration and file/data utilities can be used.


ModuleNotFoundError: No module named 'torch'

In [ ]:
# Automatically detect device (GPU > MPS > CPU)
device = "cuda" if torch.cuda.is_available() else "mps" if torch.backends.mps.is_available() else "cpu"

print(f"Using device: {device}")

In [ ]:
# # Load the model and tokenizer
model_name = "EleutherAI/pythia-70m"
tokenizer = AutoTokenizer.from_pretrained(model_name)

# Load model & move to device
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    torch_dtype=torch.float16 if device == "cuda" else torch.float32,
    device_map="auto"
)


In [ ]:
# tokenize prompt
tokenizer.pad_token_id = tokenizer.eos_token_id
inputs = tokenizer("What is AI?", return_tensors="pt",padding=True, truncation=True,max_length=100)

# generation
outputs = model.generate(**inputs.to(device))
print(tokenizer.decode(outputs[0], skip_special_tokens=True))

In [ ]:
# create a question and answer set in a list with dictionary

from datasets import Dataset

QA_list = []

with open ('QA.txt') as doc:
    Q_counter = 0
    for line in doc:
        if 'Question' in line:
            Q_dict = {'question':line.replace('Question:"',"").strip().strip('"'),'answer':''}
            QA_list.append(Q_dict)
            Q_counter+=1
        elif 'Answer' in line:
            A_counter = Q_counter-1
            modified_line = line.replace('Answer:"',"").strip().strip('"')
            QA_list[A_counter]['answer'] = modified_line
        else:
            pass

dataset = Dataset.from_list(QA_list)

# check dataset
dataset